In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import yaml, os

from utils.spark_path import (
    get_current_year_month_week_path,
    get_latest_year_month_path,
    get_latest_year_month_week_path
)

In [2]:
# ============================================================
# Spark
# ============================================================
spark = (
    SparkSession.builder
    .appName("silver_clean_restaurant")
    .master("spark://spark-master:7077")
    .config("spark.sql.adaptive.enabled", "true")
    .config("spark.sql.shuffle.partitions", "200")
    .getOrCreate()
)

Picked up JAVA_TOOL_OPTIONS: -Dfile.encoding=UTF-8
Picked up JAVA_TOOL_OPTIONS: -Dfile.encoding=UTF-8
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/02/21 02:16:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# ============================================================
# Config
# ============================================================

CONFIG_PATH = "./config.yaml"

with open(CONFIG_PATH, "r", encoding="utf-8") as f:
    cfg = yaml.safe_load(f)

ROOT = cfg["data_lake"]["root"]
LAYERS = cfg["data_lake"]["layers"]

REGION = cfg["target"]["REGION"]

# -------------------------
# INPUT
# -------------------------

restaurant_src_base = os.path.join(
    ROOT,
    LAYERS["bronze"]["domains"]["restaurant_owner"]["paths"]["parquet"]
)

restaurant_src_path = get_latest_year_month_week_path(
    spark, restaurant_src_base
)

s0_address_base = os.path.join(
    ROOT,
    LAYERS["silver"]["stages"]["s0"]["domains"]["address"]["paths"]["parquet"]
)

s0_address_path = get_latest_year_month_path(
    spark, s0_address_base
)

# -------------------------
# OUTPUT
# -------------------------

restaurant_clean_base = os.path.join(
    ROOT,
    LAYERS["silver"]["clean"]["domains"]["restaurant_clean"]["paths"]["parquet"]
)

restaurant_clean_path = get_current_year_month_week_path(
    restaurant_clean_base
)

partition_cols = LAYERS["silver"]["clean"]["domains"]["restaurant_clean"].get(
    "partition", ["region"]
)

print("[SOURCE] restaurant =", restaurant_src_path)
print("[SOURCE] address    =", s0_address_path)
print("[TARGET] restaurant =", restaurant_clean_path)
print("[CONF] partition    =", partition_cols)

[SOURCE] restaurant = /opt/spark/data/bronze/restaurant_owner/parquet/year=2026/month=02/week=03
[SOURCE] address    = /opt/spark/data/silver/s0/address/year=2026/month=02
[TARGET] restaurant = /opt/spark/data/silver/clean/restaurant/year=2026/month=02/week=03
[CONF] partition    = ['region']


In [4]:
# ============================================================
# LOAD
# ============================================================

rest_df = (
    spark.read.parquet(restaurant_src_path)
    .filter(F.col("region") == REGION)
    .select("업체명", "대표자", "소재지", "업종", "region")
)

addr_df = (
    spark.read.parquet(s0_address_path)
    .filter(F.col("region") == REGION)
    .select("PNU코드", "도로명주소", "longitude", "latitude", "region")
)


print("=== raw schema ===")
rest_df.printSchema()
print("restaurant :", rest_df.count())
print("address   :", addr_df.count())

=== raw schema ===
root
 |-- 업체명: string (nullable = true)
 |-- 대표자: string (nullable = true)
 |-- 소재지: string (nullable = true)
 |-- 업종: string (nullable = true)
 |-- region: string (nullable = true)

restaurant : 212286
address   : 1029758


In [5]:
# ============================================================
# CLEANING
# ============================================================

clean_rest_addr_df = (
    rest_df

    # 업체명 NULL 제거
    .filter(F.col("업체명").isNotNull())
    # 대표자 NULL 제거
    .filter(F.col("대표자").isNotNull())
    # 소재지 NULL 제거
    .filter(F.col("소재지").isNotNull())

    # 괄호 제거
    .withColumn("소재지", F.regexp_replace(F.col("소재지"), r"\s*\([^)]*\)", ""))

    # 쉼표 뒤 제거
    .withColumn("소재지", F.regexp_replace(F.col("소재지"), r",.*$", ""))

    # 도로명 앞 행정동 제거
    .withColumn(
        "소재지",
        F.regexp_replace(
            F.col("소재지"),
            r"\s+\S+(동|읍|면|리)\s+(?=\S+(로|길))",
            " "
        )
    )

    # 공백 정리
    .withColumn("소재지", F.trim(F.regexp_replace(F.col("소재지"), r"\s+", " ")))
)

clean_rest_df = (
    clean_rest_addr_df
    # 대표자수 계산
    .withColumn(
        "대표자수",
         F.when(
             F.col("대표자").rlike(r"외\s*\d+"),
             1 + F.regexp_extract(F.col("대표자"), r"외\s*(\d+)", 1).cast("int")
         )
         .when(
             F.col("대표자").rlike(r"[,/·]"),
             F.size(F.split(F.col("대표자"), r"\s*[,/·]\s*"))
         )
         .otherwise(F.lit(1))
    )

    # 대표자 정규화
    .withColumn("대표자", F.trim(F.col("대표자")))
    .withColumn("대표자", F.regexp_replace(F.col("대표자"), r"\s*\([^)]*\)\s*", ""))   # 괄호 제거
    .withColumn("대표자", F.regexp_replace(F.col("대표자"), r"\s*[,/].*$", ""))       # 첫 토큰만
    .withColumn("대표자", F.regexp_replace(F.col("대표자"), r"\s*외\s*\d+\s*(인|명)\s*$", ""))
    .withColumn("대표자", F.regexp_replace(F.col("대표자"), r"\s*외\s*\d+\s*$", ""))  # "외3"
    .withColumn("대표자", F.regexp_replace(F.col("대표자"), r"\s+", " "))
    .withColumn("대표자", F.trim(F.col("대표자")))

    .dropDuplicates(["업체명", "대표자", "소재지"])
)

print("restaurant :", clean_rest_df.count())
clean_rest_df.show(5, truncate=False)

restaurant : 189777


[Stage 14:======================================>                   (2 + 1) / 3]

+-----------+------+---------------------------------------+----------+------+--------+
|업체명     |대표자|소재지                                 |업종      |region|대표자수|
+-----------+------+---------------------------------------+----------+------+--------+
|#180카페   |양*경 |경기도 이천시 어재연로 3               |휴게음식점|경기  |1       |
|#갓떡뽀끼  |최*희 |경기도 의정부시 금오동 343-1 지상1층   |일반음식점|경기  |1       |
|#그술집    |양*영 |경기도 성남시 분당구 매화로37번길 16   |일반음식점|경기  |1       |
|#깐부랭삼  |신*민 |경기도 남양주시 다산중앙로146번길 12-14|일반음식점|경기  |1       |
|#깡패소곱창|공*균 |경기도 동두천시 거북마루로 4           |일반음식점|경기  |1       |
+-----------+------+---------------------------------------+----------+------+--------+
only showing top 5 rows


In [6]:
# ============================================================
# Address Matching
# ============================================================

joined_df = (
    clean_rest_df.alias("r")
    .join(
        addr_df.alias("a"),
        F.col("r.소재지") == F.col("a.도로명주소"),
        "left"
    )
)

total = joined_df.count()
matched = joined_df.filter(F.col("PNU코드").isNotNull()).count()

print("total     :", total)
print("matched   :", matched)
print("unmatched :", total - matched)
print("match rate:", matched / total if total else 0)

total     : 189812
matched   : 186751
unmatched : 3061
match rate: 0.9838735169536172


In [7]:
restaurant_clean_df = (
    joined_df
    .filter(F.col("PNU코드").isNotNull())
    .select(
        F.col("r.업체명").alias("업체명"),
        F.col("r.대표자").alias("대표자"),
        F.col("r.소재지").alias("소재지"),
        F.col("a.PNU코드").alias("PNU코드"),
        F.col("a.longitude").alias("longitude"),
        F.col("a.latitude").alias("latitude"),
        F.col("r.region").alias("region")
    )
    
)

print("=== clean schema ===")
restaurant_clean_df.printSchema()
restaurant_clean_df.show(5, truncate=False)
print("clean count:", restaurant_clean_df.count())

=== clean schema ===
root
 |-- 업체명: string (nullable = true)
 |-- 대표자: string (nullable = true)
 |-- 소재지: string (nullable = true)
 |-- PNU코드: string (nullable = true)
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- region: string (nullable = true)



+-----------------------------+------+------------------------------------+-------------------+------------------+------------------+------+
|업체명                       |대표자|소재지                              |PNU코드            |longitude         |latitude          |region|
+-----------------------------+------+------------------------------------+-------------------+------------------+------------------+------+
|명인대동양평해장국 북수원본점|김*현 |경기도 수원시 장안구 경수대로 1036-2|4111112900106260001|126.99894681547408|37.307597390511155|경기  |
|수원시립노인전문요양원       |김*기 |경기도 수원시 장안구 수일로 33      |4111112900101990002|126.99525916922042|37.31266441795493 |경기  |
|엄마닭                       |안*지 |경기도 수원시 장안구 파장로 37      |4111112900104950037|126.99642099521877|37.305147367330235|경기  |
|주당                         |손*정 |경기도 수원시 장안구 파장로 80      |4111112900105800006|126.99222640897464|37.30641023758697 |경기  |
|썬                           |김*인 |경기도 수원시 장안구 파장로 96      |4111112900105720056|126.9909401657259 |37.30749952710708 |경

clean count: 186751


In [8]:
# ============================================================
# SAVE (silver/clean/restaurant)
# ============================================================
(
    restaurant_clean_df
    .write
    .mode("overwrite")
    .partitionBy(*partition_cols)
    .parquet(restaurant_clean_path)
)

print("✅ saved:", restaurant_clean_path)

✅ saved: /opt/spark/data/silver/clean/restaurant/year=2026/month=02/week=03


In [9]:
spark.stop()